<a href="https://colab.research.google.com/github/starkdg/pyConvnetPhash/blob/master/train_cae_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive/')


In [0]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

model_dir = "/gdrive/My Drive/models"
module_url = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1"
mobilenetv2_module_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/2"

module = hub.Module(mobilenetv2_module_url)
target_height, target_width = hub.get_expected_image_size(module)

input_dims = 1792

training_files_dir = "/gdrive/My Drive/imageset/train"
validation_files_dir = "/gdrive/My Drive/imageset/validation"
testing_files_dir = "/gdrive/My Drive/imageset/test"

batch_size = 10
epochs = 10
steps = 2000
alpha = 0.0001
lambda_reg = 0.001
normalization_constant = 5.0

In [0]:
class ContractiveAutoencoder(object):
  def __init__(self, n_input, n_hidden, lambda_reg = 0.001, optimizer = tf.train.AdamOptimizer(), scope=None):
    self.n_input = n_input
    self.n_hidden = n_hidden
    
    with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
      self.weights = self._initialize_weights()
      
      # model
      reg_term = tf.constant(lambda_reg, tf.float32)
      self.x = tf.placeholder(tf.float32, [None, n_input])
      self.h = tf.nn.sigmoid(tf.add(tf.matmul(self.x,self.weights['w1']), self.weights['b1']), name='hidden')
      self.y = tf.identity(tf.add(tf.matmul(self.h, self.weights['w2']), self.weights['b2']), name='reconstructed')
    
      # Jacobian norm
      dhi = tf.square(tf.multiply(self.h, tf.subtract(1., self.h)))                  # N x n_hidden
      dwj = tf.reduce_sum(tf.square(self.weights['w2']), axis=1, keepdims=True)      # n_hidden x n_input => n_hidden x 1
      jnorm = tf.matmul(dhi, dwj, name="jnorm")                                      # N x 1 
      self.jnorm_mean = tf.reduce_mean(jnorm)
  
      # cost 
      cost = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=self.x, logits=self.y), axis=1, keepdims=True)
      self.avg_cost = tf.reduce_mean(tf.add(cost, tf.multiply(reg_term, jnorm)))

      with tf.variable_scope('opt', reuse=tf.AUTO_REUSE):
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
          self.train_op = optimizer.minimize(self.avg_cost)
     
      self.init = tf.global_variables_initializer()
      self.sess = tf.Session()
      self.sess.run(self.init)
      
  def _initialize_weights(self):
    all_weights = dict()
    all_weights['w1'] = tf.get_variable("w1", shape=[self.n_input, self.n_hidden], trainable=True, initializer=tf.contrib.layers.xavier_initializer())
    all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype = tf.float32), trainable=True)
    all_weights['w2'] = tf.transpose(all_weights['w1'], name='w2')
    all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype = tf.float32), trainable=True)
    return all_weights

  def partial_fit(self, X):
    avg_cost, jnorm_mean, opt = self.sess.run((self.avg_cost, self.jnorm_mean, self.train_op), feed_dict={self.x: X})
    return avg_cost, jnorm_mean

  def compute_cost(self, X):
    avg_cost, jnorm_mean = self.sess.run((self.avg_cost, self.jnorm_mean), feed_dict={self.x: X})
    return avg_cost, jnorm_mean

  def transform(self, X):
    encoded = self.sess.run(self.h, feed_dict={self.x: X})
    return encoded

  def reconstruct(self, X):
    return self.sess.run(self.y, feed_dict={self.x: X})

  def getWeights(self):
    return self.sess.run(self.weights['w1'])

  def getBiases(self):
    return self.sess.run(self.weights['b1'])

  def save_weights(self, path, global_step):
    saver = tf.train.Saver({'w1': self.weights['w1'],
                            'b1': self.weights['b1'],
                            'b2': self.weights['b2']}, max_to_keep=2)
    saver.save(self.sess, path, global_step=global_step)

  def load_weights(self, path):
    saver = tf.train.Saver({'w1': self.weights['w1'],
                            'b1': self.weights['b1'],
                            'b2': self.weights['b2']})
    saver.restore(self.sess, path)


In [0]:
def get_tfrecord_files(path):
  files = []
  for entry in os.scandir(path):
    if entry.is_file() and entry.name.endswith('.tfrecord'):
             files.append(entry.path)
  return files
  
  
def _parse_example(example):
  features = {'height': tf.FixedLenFeature([], tf.int64),
              'width': tf.FixedLenFeature([], tf.int64),
              'image_raw': tf.FixedLenFeature([], tf.string)}
  parsed_features = tf.parse_single_example(example, features)
  img = parsed_features['image_raw']
  img = tf.io.decode_raw(parsed_features['image_raw'], tf.uint8)
  height = tf.cast(parsed_features['height'], tf.int32)
  width = tf.cast(parsed_features['width'], tf.int32)
  img_reshaped = tf.manip.reshape(img, [height, width, 3])
  imgfl = tf.image.convert_image_dtype(img_reshaped, dtype=tf.float32)
  imgfl = tf.expand_dims(imgfl, 0)
  img_resized = tf.image.resize_bicubic(imgfl, [target_height, target_width])
  img_resized = tf.squeeze(img_resized, 0)

  return img_resized


def input_function(path, batch_size=1, num_epochs=None, shuffle=False):
  tfrecords = get_tfrecord_files(path)
  dataset = tf.data.TFRecordDataset(tfrecords)
  dataset = dataset.map(_parse_example)
  if (shuffle):
    dataset = dataset.shuffle(10000)
  dataset = dataset.batch(batch_size).repeat(num_epochs)
  iterator = dataset.make_initializable_iterator()
  return iterator


def normalize(x):
  norm_x = (x - np.amin(x)) / (np.amax(x) - np.amin(x))
  return norm_x

In [0]:
def train_autoenc_model(training_files_dir,
                        validation_files_dir,
                        testing_files_dir,
                        batch_size, epochs, steps, alpha,
                        lambda_reg):
  period_size = 100
  training_iter = input_function(training_files_dir, batch_size)
  training_images = training_iter.get_next()
  training_features = module(training_images)
    
  validation_iter = input_function(validation_files_dir, batch_size)
  validation_images = validation_iter.get_next()
  validation_features = module(validation_images)
    
  testing_iter = input_function(testing_files_dir, 100)
  testing_images = testing_iter.get_next()
  testing_features = module(testing_images)
     
  opt = tf.train.AdamOptimizer(learning_rate=alpha) 
  
  cae_n = 3              # layer number to train
  saved_n = 1            # save iteration (global step  in saved dir)
  dim_from = 512         # from dimension
  dim_to = 256           # to dimension
  
  # uncomment each layer to train each layer 
  cae1 = ContractiveAutoencoder(input_dims, 1024, lambda_reg=lambda_reg, optimizer=opt, scope='cae1')
  cae2 = ContractiveAutoencoder(1024, 512, lambda_reg=lambda_reg, optimizer=opt, scope='cae2')
  cae3 = ContractiveAutoencoder(512, 256, lambda_reg=lambda_reg, optimizer=opt, scope='cae3')
  
  # set to last layer 
  cae = cae3
  
  save_dir_str = '/gdrive/My Drive/models/cae{0}/cae.chp-{1}'
  save_dir1 =  save_dir_str.format(1, saved_n)
  save_dir2 =  save_dir_str.format(2, saved_n)
  save_dir3 =  save_dir_str.format(3, saved_n)
  save_cae_to_dir = "/gdrive/My Drive/models/cae{0}/cae.chp".format(cae_n)
  training_cae_title = "CAE-{0}to{1}".format(dim_from, dim_to)
  
  try:
    # load all except last layer
    cae1.load_weights(save_dir1)
    print("load weights for cae-1")
    cae2.load_weights(save_dir2)
    print("load weights for cae-2")
    cae3.load_weights(save_dir3)
    print("load weights for cae-3")
  except ValueError:
    print("Train CAE layer ", cae_n)
          
  sess = tf.Session()
  sess.run(tf.global_variables_initializer())

  fig_n = 1
  train_recon_losses = []
  valid_recon_losses = []
  valid_jnorm_losses = []
    
  for i in range(epochs):
    sess.run([training_iter.initializer, validation_iter.initializer])
    iteration = 0
    total_train_recon_cost = 0
    total_valid_recon_cost = 0
    total_train_jnorm_cost = 0
    total_valid_jnorm_cost = 0
        
    while True:
      try:
        Xtrain = sess.run(training_features)
        Xtrain = normalize(Xtrain)
        Xtrain = cae1.transform(Xtrain)
        Xtrain = cae2.transform(Xtrain)
        # last layer to be trained
        train_cost, train_jnorm = cae.partial_fit(Xtrain)      
        if (iteration % period_size == 0):
          Xvalid = sess.run(validation_features)
          Xvalid = normalize(Xvalid)
          Xvalid = cae1.transform(Xvalid)
          Xvalid = cae2.transform(Xvalid)
          # check validation cost for last layer
          valid_cost, valid_jnorm = cae.compute_cost(Xvalid)
          total_train_recon_cost += train_cost
          total_valid_recon_cost += valid_cost
          total_train_jnorm_cost += train_jnorm
          total_valid_jnorm_cost += valid_jnorm
  
        iteration = iteration + 1
      except tf.errors.OutOfRangeError:
        break
      if (iteration > steps):
        break
                
    
    steps_taken = iteration//period_size
    avg_train_recon_loss = total_train_recon_cost/steps_taken
    avg_valid_recon_loss = total_valid_recon_cost/steps_taken
    avg_valid_jnorm_loss = total_valid_jnorm_cost/steps_taken
    
    print("epoch {0} training cost = {1} valid. cost= {2} (jnorm = {3})".format(
       i+1, avg_train_recon_loss, avg_valid_recon_loss, avg_valid_jnorm_loss))
    
             
    train_recon_losses.append(avg_train_recon_loss)
    valid_recon_losses.append(avg_valid_recon_loss)
    valid_jnorm_losses.append(avg_valid_jnorm_loss)


  cae.save_weights(save_cae_to_dir, global_step=saved_n)
  plt.figure(fig_n)
  plt.plot(train_recon_losses)
  plt.plot(valid_recon_losses)
  plt.plot(valid_jnorm_losses)
  
  plt.title("{0} Autoencoder Training Losses".format(training_cae_title))
  plt.legend(['train loss', 'valid. loss', 'jnorm loss'], loc='upper right')
  plt.xlabel("epochs")
  plt.ylabel("cost")
  plt.show()

  print("run test on 100 images")
  sess.run([testing_iter.initializer])
  Xtest = sess.run(testing_features)
  Xtest = normalize(Xtest)
  Xtest = cae1.transform(Xtest)
  Xtest = cae2.transform(Xtest)
  # compute cost for test set
  test_cost, test_jnorm = cae.compute_cost(Xtest)
  print("test cost = {0}, jnorm = {1}".format(test_cost, test_jnorm))
  sess.close()

  fig_n += 1
    
  w1 = cae.getWeights()
  b1 = cae.getBiases()
  plt.figure(fig_n)
  plt.hist((w1.ravel()), bins=64, histtype='bar', stacked=True)
  plt.legend(['w1'], loc='upper right')
  plt.title('histogram of weights for last layer')

  print("w1:")
  print("mean: ", np.mean(w1))
  print("min: ", np.amin(w1))
  print("max: ", np.amax(w1))
  print("std: ", np.std(w1))
    
  fig_n += 1
  plt.figure(fig_n)
  plt.hist((b1.ravel()), bins=100, histtype='bar', stacked=True)
  plt.legend(['b1'], loc='upper right')
  plt.title('histogram of biases for last layer')
  
  print("b1:")
  print("mean: ", np.mean(b1))
  print("min: ", np.amin(b1))
  print("max: ", np.amax(b1))
  print("std: ", np.std(b1))
  
  


In [0]:
print("Train autoencoder")
print("training files: ", training_files_dir)
print("validation files: ", validation_files_dir)
print("testing files: ", testing_files_dir)
print("alpha: ", alpha)
print("lambda reg: ", lambda_reg)
print("batch size: ", batch_size)
print("epochs: ", epochs)
print("steps: ", steps)

train_autoenc_model(training_files_dir,
                    validation_files_dir,
                    testing_files_dir,
                    batch_size, epochs, steps,
                    alpha, lambda_reg)
